# Parser for RateBeer

In this notebook, we are parsing the HTML pages from the RateBeer website.

In [ ]:
import os
os.chdir('..')

In [ ]:
from classes.parser import *

import pandas as pd
import numpy as np
import datetime
import time
import html
import gzip
import re
import os

data_folder = '../data/'

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
parser = Parser(data_folder)

# Parse the breweries from the places

In [ ]:
parser.parse_breweries_from_places()

# Parse the beers from the breweries

In [ ]:
parser.parse_brewery_files()

# Parse the beer files for information

In [ ]:
parser.parse_beer_files_for_information()

# Parse the beer files for the reviews

In [ ]:
parser.parse_beer_files_for_reviews()

# Get the users from the ratings

In [ ]:
parser.get_users_from_ratings()

# Parse the user files for information

In [ ]:
parser.parse_all_users()